In [8]:
"""
Wings R Us Recommendation System - Optimized Version
====================================================

This script implements an optimized recommendation system for Wings R Us restaurant
that efficiently handles large datasets and generates high-quality recommendations.

Key Features:
- Efficient data processing for large datasets (1.4M+ orders)
- Advanced item similarity analysis using TF-IDF and cosine similarity
- Business rule-based recommendations for category complementarity
- Customer behavior analysis and personalization
- Optimized for Recall@3 evaluation metric
"""

'\nWings R Us Recommendation System - Optimized Version\n====================================================\n\nThis script implements an optimized recommendation system for Wings R Us restaurant\nthat efficiently handles large datasets and generates high-quality recommendations.\n\nKey Features:\n- Efficient data processing for large datasets (1.4M+ orders)\n- Advanced item similarity analysis using TF-IDF and cosine similarity\n- Business rule-based recommendations for category complementarity\n- Customer behavior analysis and personalization\n- Optimized for Recall@3 evaluation metric\n'

# Import libraries

In [27]:
# Import libraries
import pandas as pd
import numpy as np
import json
import re
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import os
warnings.filterwarnings('ignore')

# Set Display options

# Data Analyzer: Analyzes order and customer data to extract patterns and preferences efficiently


In [11]:
class DataAnalyzer:
    """Analyzes large datasets efficiently"""
    
    @staticmethod
    def extract_items_from_json(json_str):
        """Extract item names from JSON order data"""
        try:
            data = json.loads(json_str)
            items = []
            for order in data.get('orders', []):
                for item_detail in order.get('item_details', []):
                    item_name = item_detail.get('item_name', '')
                    if item_name and not item_name.startswith('Order Memo'):
                        items.append(item_name)
            return items
        except:
            return []
    
    @staticmethod
    def analyze_order_patterns(order_data, sample_size=10000):
        """Analyze order patterns from large order dataset"""
        print("Analyzing order patterns...")
        
        # Sample the data for analysis
        sample_data = order_data.sample(n=min(sample_size, len(order_data)), random_state=42)
        
        all_items = []
        item_combinations = []
        item_frequencies = Counter()
        
        for _, row in sample_data.iterrows():
            items = DataAnalyzer.extract_items_from_json(row['ORDERS'])
            if items:
                all_items.extend(items)
                item_frequencies.update(items)
                
                # Store item combinations (2-3 items)
                if len(items) >= 2:
                    for i in range(len(items)):
                        for j in range(i+1, min(i+3, len(items))):
                            item_combinations.append(tuple(sorted([items[i], items[j]])))
        
        return {
            'item_frequencies': dict(item_frequencies.most_common(100)),
            'item_combinations': Counter(item_combinations),
            'total_orders': len(sample_data)
        }
    
    @staticmethod
    def analyze_customer_behavior(customer_data, order_data, sample_size=10000):
        """Analyze customer behavior patterns"""
        print("Analyzing customer behavior...")
        
        # Sample data for analysis
        sample_orders = order_data.sample(n=min(sample_size, len(order_data)), random_state=42)
        
        customer_preferences = defaultdict(lambda: defaultdict(int))
        customer_types = {}
        
        # Get customer types
        for _, row in customer_data.iterrows():
            customer_types[row['CUSTOMER_ID']] = row['CUSTOMER_TYPE']
        
        # Analyze customer preferences
        for _, row in sample_orders.iterrows():
            customer_id = row['CUSTOMER_ID']
            customer_type = customer_types.get(customer_id, 'Unknown')
            items = DataAnalyzer.extract_items_from_json(row['ORDERS'])
            
            for item in items:
                customer_preferences[customer_type][item] += 1
        
        return dict(customer_preferences)


# Item Categorizer: Categorizes items and extracts features from item names for recommendation systems

In [12]:
class ItemCategorizer:
    """Advanced item categorization and feature extraction"""
    
    @staticmethod
    def categorize_item(item):
        """Categorize items into main categories with enhanced logic"""
        if pd.isna(item):
            return 'Unknown'
        
        item_str = str(item).lower()
        
        # Wings categories
        if 'wings' in item_str:
            if 'spicy' in item_str:
                return 'Spicy Wings'
            elif 'grilled' in item_str:
                return 'Grilled Wings'
            elif 'mixed' in item_str:
                return 'Mixed Wings'
            elif 'feast' in item_str or 'family' in item_str:
                return 'Family Wings'
            else:
                return 'Wings'
        
        # Chicken products
        elif any(x in item_str for x in ['strips', 'tenders', 'crispy']):
            return 'Chicken Strips/Tenders'
        elif 'sub' in item_str:
            return 'Chicken Sub'
        
        # Sides and accompaniments
        elif 'fries' in item_str:
            if 'buffalo' in item_str:
                return 'Buffalo Fries'
            elif 'cheese' in item_str:
                return 'Cheese Fries'
            elif 'voodoo' in item_str:
                return 'Voodoo Fries'
            else:
                return 'Fries'
        elif 'corn' in item_str:
            return 'Fried Corn'
        elif 'veggie' in item_str or 'carrot' in item_str:
            return 'Veggies'
        
        # Dips and sauces
        elif any(x in item_str for x in ['dip', 'ranch', 'blue cheese', 'honey mustard']):
            return 'Dips'
        
        # Beverages
        elif any(x in item_str for x in ['soda', 'drink', 'oz']):
            return 'Beverages'
        
        # Desserts
        elif any(x in item_str for x in ['cake', 'chocolate']):
            return 'Desserts'
        
        # Special items
        elif 'flavor platter' in item_str:
            return 'Flavor Platter'
        elif 'add' in item_str:
            return 'Add-On Wings'
        
        else:
            return 'Other'
    
    @staticmethod
    def extract_item_features(item):
        """Extract detailed features from item names"""
        if pd.isna(item):
            return {}
        
        item_str = str(item).lower()
        features = {}
        
        # Size features
        if 'large' in item_str:
            features['size'] = 'large'
        elif 'regular' in item_str:
            features['size'] = 'regular'
        elif 'medium' in item_str:
            features['size'] = 'medium'
        
        # Quantity features
        pc_match = re.search(r'(\d+)\s*pc', item_str)
        if pc_match:
            features['pieces'] = int(pc_match.group(1))
        
        # Combo features
        if 'combo' in item_str:
            features['is_combo'] = True
        
        # Spice level
        if 'spicy' in item_str:
            features['spice_level'] = 'spicy'
        elif 'mild' in item_str:
            features['spice_level'] = 'mild'
        
        # Special features
        if 'feast' in item_str:
            features['is_feast'] = True
        if 'family' in item_str:
            features['is_family'] = True
        
        return features

# Advanced Recommender: Generates advanced item recommendations using multiple strategies and similarity analysis

In [13]:
class AdvancedRecommender:
    """Advanced recommendation system with multiple strategies"""
    
    def __init__(self):
        self.item_similarity_matrix = None
        self.item_frequency = {}
        self.category_complementarity = {}
        self.business_rules = {}
        self.customer_preferences = {}
        self.item_combinations = {}
        self.item_to_index = {}
        
    def fit(self, test_data, order_analysis, customer_analysis):
        """Train the recommendation system with comprehensive data"""
        print("Training advanced recommendation system...")
        
        # Set up data
        self.item_frequency = order_analysis['item_frequencies']
        self.item_combinations = order_analysis['item_combinations']
        self.customer_preferences = customer_analysis
        
        # Calculate category complementarity
        self._calculate_advanced_category_complementarity(test_data)
        
        # Create enhanced business rules
        self._create_enhanced_business_rules()
        
        # Build advanced similarity matrix
        self._build_advanced_similarity_matrix(test_data)
        
        print("Advanced training completed!")
    
    def _calculate_advanced_category_complementarity(self, data):
        """Calculate advanced category complementarity patterns"""
        print("Calculating category complementarity...")
        
        # Analyze actual order patterns
        category_pairs = defaultdict(int)
        total_orders = 0
        
        for _, row in data.iterrows():
            items = [row['item1'], row['item2'], row['item3']]
            items = [item for item in items if pd.notna(item)]
            
            if len(items) >= 2:
                total_orders += 1
                categories = [ItemCategorizer.categorize_item(item) for item in items]
                
                for i in range(len(categories)):
                    for j in range(i+1, len(categories)):
                        pair = tuple(sorted([categories[i], categories[j]]))
                        category_pairs[pair] += 1
        
        # Create complementarity matrix
        self.category_complementarity = {}
        for (cat1, cat2), count in category_pairs.items():
            if count >= total_orders * 0.01:  # At least 1% of orders
                if cat1 not in self.category_complementarity:
                    self.category_complementarity[cat1] = []
                if cat2 not in self.category_complementarity:
                    self.category_complementarity[cat2] = []
                
                self.category_complementarity[cat1].append(cat2)
                self.category_complementarity[cat2].append(cat1)
    
    def _create_enhanced_business_rules(self):
        """Create enhanced business rules based on data analysis"""
        print("Creating enhanced business rules...")
        
        # Popular items by category
        popular_items = defaultdict(list)
        for item, freq in sorted(self.item_frequency.items(), key=lambda x: x[1], reverse=True):
            category = ItemCategorizer.categorize_item(item)
            if len(popular_items[category]) < 5:  # Top 5 per category
                popular_items[category].append(item)
        
        self.business_rules = {
            'missing_drink': popular_items['Beverages'][:3],
            'missing_dip': popular_items['Dips'][:3],
            'missing_side': popular_items['Fries'] + popular_items['Fried Corn'][:2],
            'missing_wings': popular_items['Wings'] + popular_items['Spicy Wings'][:2],
            'missing_chicken': popular_items['Chicken Strips/Tenders'] + popular_items['Chicken Sub'][:2],
            'missing_veggies': popular_items['Veggies'][:2]
        }
    
    def _build_advanced_similarity_matrix(self, data):
        """Build advanced item similarity matrix"""
        print("Building similarity matrix...")
        
        # Create comprehensive item descriptions
        item_descriptions = {}
        for col in ['item1', 'item2', 'item3']:
            for item in data[col].dropna().unique():
                if item not in item_descriptions:
                    category = ItemCategorizer.categorize_item(item)
                    features = ItemCategorizer.extract_item_features(item)
                    
                    desc_parts = [category]
                    if 'size' in features:
                        desc_parts.append(features['size'])
                    if 'spice_level' in features:
                        desc_parts.append(features['spice_level'])
                    if features.get('is_combo', False):
                        desc_parts.append('combo')
                    if features.get('is_feast', False):
                        desc_parts.append('feast')
                    if features.get('is_family', False):
                        desc_parts.append('family')
                    
                    item_descriptions[item] = ' '.join(desc_parts)
        
        # Create TF-IDF vectors
        if item_descriptions:
            try:
                vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
                items = list(item_descriptions.keys())
                descriptions = list(item_descriptions.values())
                
                tfidf_matrix = vectorizer.fit_transform(descriptions)
                self.item_similarity_matrix = cosine_similarity(tfidf_matrix)
                self.item_to_index = {item: idx for idx, item in enumerate(items)}
                print(f"Similarity matrix built for {len(items)} items")
            except Exception as e:
                print(f"Warning: Could not build similarity matrix: {e}")
                self.item_similarity_matrix = None
    
    def recommend(self, cart_items, customer_type='Guest', n_recommendations=3):
        """Generate advanced recommendations for a given cart"""
        recommendations = []
        
        # Remove None/NaN values
        cart_items = [item for item in cart_items if pd.notna(item)]
        
        if not cart_items:
            # If no items, return most popular items
            top_items = sorted(self.item_frequency.items(), key=lambda x: x[1], reverse=True)
            return [item for item, _ in top_items[:n_recommendations]]
        
        # Strategy 1: Business rules based on missing categories
        missing_categories = self._identify_missing_categories(cart_items)
        rule_recommendations = self._apply_enhanced_business_rules(missing_categories)
        recommendations.extend(rule_recommendations)
        
        # Strategy 2: Customer type preferences
        if customer_type in self.customer_preferences:
            customer_recs = self._get_customer_type_recommendations(cart_items, customer_type, n_recommendations)
            recommendations.extend(customer_recs)
        
        # Strategy 3: Item combination analysis
        combination_recs = self._get_combination_recommendations(cart_items, n_recommendations)
        recommendations.extend(combination_recs)
        
        # Strategy 4: Similarity-based recommendations
        if self.item_similarity_matrix is not None:
            similarity_recs = self._get_advanced_similarity_recommendations(cart_items, n_recommendations)
            recommendations.extend(similarity_recs)
        
        # Strategy 5: Popularity-based recommendations
        popularity_recs = self._get_enhanced_popularity_recommendations(cart_items, n_recommendations)
        recommendations.extend(popularity_recs)
        
        # Remove duplicates and cart items, limit to n_recommendations
        final_recommendations = []
        seen = set(cart_items)
        
        for rec in recommendations:
            if rec not in seen and rec not in final_recommendations:
                final_recommendations.append(rec)
                seen.add(rec)
                if len(final_recommendations) >= n_recommendations:
                    break
        
        # If we don't have enough, add popular items
        while len(final_recommendations) < n_recommendations:
            for item, _ in sorted(self.item_frequency.items(), key=lambda x: x[1], reverse=True):
                if item not in seen:
                    final_recommendations.append(item)
                    seen.add(item)
                    break
        
        return final_recommendations[:n_recommendations]
    
    def _identify_missing_categories(self, cart_items):
        """Identify which categories are missing from the cart"""
        cart_categories = set()
        for item in cart_items:
            category = ItemCategorizer.categorize_item(item)
            cart_categories.add(category)
        
        all_categories = set(self.category_complementarity.keys())
        missing_categories = all_categories - cart_categories
        
        return list(missing_categories)
    
    def _apply_enhanced_business_rules(self, missing_categories):
        """Apply enhanced business rules to generate recommendations"""
        recommendations = []
        
        for category in missing_categories:
            if category == 'Beverages':
                recommendations.extend(self.business_rules['missing_drink'])
            elif category == 'Dips':
                recommendations.extend(self.business_rules['missing_dip'])
            elif category in ['Fries', 'Buffalo Fries', 'Cheese Fries', 'Voodoo Fries']:
                recommendations.extend(self.business_rules['missing_side'])
            elif 'Wings' in category:
                recommendations.extend(self.business_rules['missing_wings'])
            elif category in ['Chicken Strips/Tenders', 'Chicken Sub']:
                recommendations.extend(self.business_rules['missing_chicken'])
            elif category == 'Veggies':
                recommendations.extend(self.business_rules['missing_veggies'])
        
        return recommendations
    
    def _get_customer_type_recommendations(self, cart_items, customer_type, n_recommendations):
        """Get recommendations based on customer type preferences"""
        if customer_type not in self.customer_preferences:
            return []
        
        customer_items = self.customer_preferences[customer_type]
        cart_categories = {ItemCategorizer.categorize_item(item) for item in cart_items}
        
        # Find popular items in complementary categories
        recommendations = []
        for item, freq in sorted(customer_items.items(), key=lambda x: x[1], reverse=True):
            if len(recommendations) >= n_recommendations:
                break
            
            item_category = ItemCategorizer.categorize_item(item)
            if item_category not in cart_categories and item not in cart_items:
                recommendations.append(item)
        
        return recommendations
    
    def _get_combination_recommendations(self, cart_items, n_recommendations):
        """Get recommendations based on item combination analysis"""
        recommendations = []
        
        # Look for items that are commonly ordered with current cart items
        for i in range(len(cart_items)):
            for j in range(i+1, len(cart_items)):
                pair = tuple(sorted([cart_items[i], cart_items[j]]))
                if pair in self.item_combinations:
                    # Find other items commonly ordered with this pair
                    for other_pair, count in self.item_combinations.items():
                        if len(recommendations) >= n_recommendations:
                            break
                        
                        # Check if this pair shares an item with our cart
                        shared_items = set(pair) & set(other_pair)
                        if shared_items and len(shared_items) < len(other_pair):
                            # Add the non-shared item
                            for item in other_pair:
                                if item not in cart_items and item not in recommendations:
                                    recommendations.append(item)
                                    break
        
        return recommendations
    
    def _get_advanced_similarity_recommendations(self, cart_items, n_recommendations):
        """Get recommendations based on advanced item similarity"""
        recommendations = []
        
        for item in cart_items:
            if item in self.item_to_index:
                idx = self.item_to_index[item]
                similarities = self.item_similarity_matrix[idx]
                
                # Get top similar items
                similar_indices = np.argsort(similarities)[::-1][1:6]  # Top 5 similar
                for sim_idx in similar_indices:
                    similar_item = list(self.item_to_index.keys())[list(self.item_to_index.values()).index(sim_idx)]
                    if similar_item not in cart_items and similar_item not in recommendations:
                        recommendations.append(similar_item)
                        if len(recommendations) >= n_recommendations:
                            break
        
        return recommendations
    
    def _get_enhanced_popularity_recommendations(self, cart_items, n_recommendations):
        """Get enhanced popularity-based recommendations"""
        cart_categories = {ItemCategorizer.categorize_item(item) for item in cart_items}
        
        # Find complementary popular items
        complementary_items = []
        for category in cart_categories:
            if category in self.category_complementarity:
                for comp_category in self.category_complementarity[category]:
                    # Find popular items in complementary category
                    for item, freq in sorted(self.item_frequency.items(), key=lambda x: x[1], reverse=True):
                        if ItemCategorizer.categorize_item(item) == comp_category and item not in cart_items:
                            complementary_items.append(item)
                            break
        
        return complementary_items

# Data loading 

In [14]:
def data_loading():
    print("\n" + "=" * 50)
    print("SECTION 1: DATA LOADING")
    print("=" * 50)
    
    try:
        # Load test data
        test_data = pd.read_csv('test_data_question.csv')
        print(f"Test data loaded: Shape = {test_data.shape}")
        print("Sample Test Data (First 2 Rows):")
        print(test_data.head(2))
        
        # Load store data
        store_data = pd.read_csv('store_data.csv')
        print(f"Store data loaded: Shape = {store_data.shape}")
        print("Sample Store Data (First 2 Rows):")
        print(store_data.head(2))
        
        # Load customer data
        customer_data = pd.read_csv('customer_data.csv')
        print(f"Customer data loaded: Shape = {customer_data.shape}")
        print("Sample Customer Data (First 2 Rows):")
        print(customer_data.head(2))
        
        # Load order data
        order_data = pd.read_csv('order_data.csv')
        print(f"Order data loaded: Shape = {order_data.shape}")
        print("Sample Order Data (First 2 Rows):")
        print(order_data.head(2))
        
        return test_data, store_data, customer_data, order_data
    
    except FileNotFoundError as e:
        print(f"Error loading data: {e}")
        print("Please ensure all CSV files are in the current directory.")
        raise

# Data Transformation

In [15]:
def data_transformation(test_data, customer_data, order_data):
    print("\n" + "=" * 50)
    print("SECTION 2: DATA TRANSFORMATION (ANALYSIS & CATEGORIZATION)")
    print("=" * 50)
    
    # Analyze order patterns
    order_analysis = DataAnalyzer.analyze_order_patterns(order_data, sample_size=50000)
    print(f"Analyzed {order_analysis['total_orders']} orders")
    print(f"Found {len(order_analysis['item_frequencies'])} unique items")
    print(f"Analyzed {len(order_analysis['item_combinations'])} item combinations")
    print("Top 5 Item Frequencies:")
    for item, freq in list(order_analysis['item_frequencies'].items())[:5]:
        print(f"  - {item}: {freq}")
    
    # Analyze customer behavior
    customer_analysis = DataAnalyzer.analyze_customer_behavior(customer_data, order_data, sample_size=50000)
    print(f"Analyzed customer preferences for {len(customer_analysis)} customer types")
    print("Sample Customer Preferences (First Type):")
    first_type = next(iter(customer_analysis))
    print(f"Type '{first_type}': Top 3 Items = {dict(list(customer_analysis[first_type].items())[:3])}")
    
    # Apply item categorization to test data
    for i in range(1, 4):
        col = f'item{i}'
        cat_col = f'item{i}_category'
        test_data[cat_col] = test_data[col].apply(ItemCategorizer.categorize_item)
    print("Item categorization completed")
    print("Sample Categorized Test Data (First 2 Rows):")
    print(test_data[['item1', 'item1_category', 'item2', 'item2_category', 'item3', 'item3_category']].head(2))
    
    return test_data, order_analysis, customer_analysis

# Model Training

In [16]:
def model_training(test_data, order_analysis, customer_analysis):
    print("\n" + "=" * 50)
    print("SECTION 3: MODEL TRAINING")
    print("=" * 50)
    
    recommender = AdvancedRecommender()
    recommender.fit(test_data, order_analysis, customer_analysis)
    
    print(f"System trained with {len(recommender.item_frequency)} unique items")
    print(f"Category complementarity: {len(recommender.category_complementarity)} categories")
    print("Sample Category Complementarity:")
    for cat, comps in list(recommender.category_complementarity.items())[:3]:
        print(f"{cat}: {comps[:3]}")  # Show first 3 complements
    
    return recommender

# Validation

In [17]:
def validation(recommender):    
    print("\n" + "=" * 50)
    print("SECTION 4: VALIDATION (TESTING WITH SAMPLE CARTS)")
    print("=" * 50)
    
    test_carts = [
        ['Chicken Sub Combo', 'Ranch Dip - Regular'],
        ['10 pc Spicy Wings', 'Regular Buffalo Fries'],
        ['20 pc Grilled Wings Combo', 'Ranch Dip - Large']
    ]
    
    for i, cart in enumerate(test_carts):
        print(f"\nCart {i+1}: {cart}")
        recommendations = recommender.recommend(cart, customer_type='Guest', n_recommendations=3)
        print(f"Recommendations: {recommendations}")

# Prediction

In [18]:
def prediction(recommender, test_data):
    print("\n" + "=" * 50)
    print("SECTION 5: PREDICTION (GENERATING RECOMMENDATIONS)")
    print("=" * 50)
    
    output_data = test_data.copy()
    recommendations = []
    
    for idx, row in test_data.iterrows():
        cart_items = [row['item1'], row['item2'], row['item3']]
        cart_items = [item for item in cart_items if pd.notna(item)]
        
        customer_type = row.get('CUSTOMER_TYPE', 'Guest')
        recs = recommender.recommend(cart_items, customer_type=customer_type, n_recommendations=3)
        recommendations.append(recs)
    
    # Add recommendation columns
    output_data['RECOMMENDATION_1'] = [rec[0] if len(rec) > 0 else 'No recommendation' for rec in recommendations]
    output_data['RECOMMENDATION_2'] = [rec[1] if len(rec) > 1 else 'No recommendation' for rec in recommendations]
    output_data['RECOMMENDATION_3'] = [rec[2] if len(rec) > 2 else 'No recommendation' for rec in recommendations]
    
    print(f"Generated {len(output_data)} recommendations")
    print("Sample Predictions (First 2 Rows):")
    print(output_data[['item1', 'item2', 'item3', 'RECOMMENDATION_1', 'RECOMMENDATION_2', 'RECOMMENDATION_3']].head(2))
    
    return output_data

# Output and Analysis 

In [28]:
def output_and_analysis(output_data, recommender, customer_analysis, order_analysis, store_data):
    print("\n" + "=" * 50)
    print("SECTION 6: OUTPUT AND ANALYSIS")
    print("=" * 50)
    
    # Create output directory
    if not os.path.exists('outputs'):
        os.makedirs('outputs')
    
    # Save Excel output
    excel_columns = ['CUSTOMER_ID', 'ORDER_ID', 'item1', 'item2', 'item3', 
                     'RECOMMENDATION_1', 'RECOMMENDATION_2', 'RECOMMENDATION_3']
    excel_output = output_data[excel_columns].copy()
    excel_output.to_excel('outputs/recommendation_output.xlsx', index=False)
    print("Excel output saved: outputs/recommendation_output.xlsx")
    
    # Save CSV output
    excel_output.to_csv('outputs/recommendation_output.csv', index=False)
    print("CSV output saved: outputs/recommendation_output.csv")
    
    # Most recommended items
    all_recommendations = []
    for col in ['RECOMMENDATION_1', 'RECOMMENDATION_2', 'RECOMMENDATION_3']:
        all_recommendations.extend(output_data[col].value_counts().index)
    
    rec_counts = pd.Series(all_recommendations).value_counts()
    print(f"\nTop 10 most recommended items:")
    for item, count in rec_counts.head(10).items():
        print(f"{item}: {count} times")
    
    # Category analysis
    print(f"\nCategory distribution in recommendations:")
    category_counts = defaultdict(int)
    for item in all_recommendations:
        category = ItemCategorizer.categorize_item(item)
        category_counts[category] += 1
    
    for category, count in sorted(category_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"{category}: {count} times")
    # Business impact estimation
    total_orders = len(output_data)
    avg_item_value = 8.50  # Estimated average item value
    conversion_rate = 0.15  # Estimated 15% conversion rate
    potential_revenue = total_orders * avg_item_value * conversion_rate
    
    print(f"\nBusiness Impact Analysis:")
    print(f"Total orders analyzed: {total_orders:,}")
    print(f"Potential daily revenue increase: ${potential_revenue:,.2f}")
    print(f"Potential annual revenue increase: ${potential_revenue * 365:,.2f}")
    print(f"Estimated conversion rate: {conversion_rate*100}%")
    
    # Data quality insights
    print(f"\nData Quality Insights:")
    print(f"Unique menu items: {len(recommender.item_frequency)}")
    print(f"Customer types analyzed: {len(customer_analysis)}")
    print(f"Store locations: {len(store_data)}")
    print(f"Order patterns analyzed: {order_analysis['total_orders']:,}")

# Main Function

In [30]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
print("=" * 70)
print("WINGS R US ADVANCED RECOMMENDATION SYSTEM - STEP-BY-STEP")
print("=" * 70)
test_data, store_data, customer_data, order_data = data_loading()
test_data, order_analysis, customer_analysis = data_transformation(test_data, customer_data, order_data)
recommender = model_training(test_data, order_analysis, customer_analysis)
validation(recommender)
output_data = prediction(recommender, test_data)
# Summary
print("\n" + "=" * 70)
print("ALL SECTIONS COMPLETED SUCCESSFULLY!")
print("=" * 70)
print(f"Generated {len(output_data)} recommendations")
print(f"Output files created in 'outputs/' directory")
print("Ready for deployment!")

WINGS R US ADVANCED RECOMMENDATION SYSTEM - STEP-BY-STEP

SECTION 1: DATA LOADING
Test data loaded: Shape = (1000, 10)
Sample Test Data (First 2 Rows):
   CUSTOMER_ID  STORE_NUMBER    ORDER_ID ORDER_CHANNEL_NAME  \
0    997177535          4915  9351345556            Digital   
1    345593831           949  3595377080            Digital   

  ORDER_SUBCHANNEL_NAME ORDER_OCCASION_NAME CUSTOMER_TYPE  \
0                   WWT                ToGo         Guest   
1                   WWT                ToGo    Registered   

                   item1                item2                     item3  
0      Chicken Sub Combo  Ranch Dip - Regular   10 pc Spicy Wings Combo  
1  Regular Buffalo Fries    10 pc Spicy Wings  3 pc Crispy Strips Combo  
Store data loaded: Shape = (38, 4)
Sample Store Data (First 2 Rows):
   STORE_NUMBER          CITY STATE POSTAL_CODE
0          2156     GRAPEVINE    TX       76051
1          1419  HUNTERSVILLE    NC       28078
Customer data loaded: Shape = (563346, 